CONTAGION 
NODE BLOCKING
ERDOS-RENYI GRAPH


In [1]:
import numpy as np
import random
import pandas as pd

#!pip install pyomo
#import pyomo.environ as pyEnv
#!pip install -i https://pypi.gurobi.com gurobipy
import gurobipy as gp
from gurobipy import GRB
import copy
import time

adjacency_df=pd.DataFrame(pd.read_csv(r"C:\\Users\\utku\Desktop\\bitirme\\\data\\adjaceny_df\\adjacency_df-n100-m400-run1-risky40-inf10_utku.csv"))

n = len(adjacency_df)

adjacency_df.columns=(range(1,(n+4)))
adjacency_df=adjacency_df.rename(columns={n+1:'Threshold',n+2:'IsInfected', n+3:'IsRisky' },errors='raise')

adjacency_df.index=(range(1,(n+1)))

m=((adjacency_df.loc[:,range(1,n+1)]>0).sum().sum())/2

#sets
setV = range(1,n+1)
setS = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] < 1),"IsInfected"].index.tolist()
setP = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] > 0),"IsInfected"].index.tolist()

start_time = time.time()
#parameters
wij = [[adjacency_df.loc[i,j] for i in setV] for j in setV  ]
ti = [adjacency_df.loc[i,"Threshold"] for i in setV]
Xi = [adjacency_df.loc[i,"IsRisky"] for i in setV]
#Blocking_Capacity = int((n) / 10) 
Blocking_Capacity = int((n) / 10)*2

#model
Math_Model = gp.Model("Math_Model")

#variables
Yi = Math_Model.addVars(setV,vtype=GRB.BINARY,name="Yi") #Bunun yerine SetS'de tanımlı olabilir!!!!
Ii = Math_Model.addVars(setV,vtype=GRB.BINARY, obj=Xi, name="Ii")
Multi= Math_Model.addVars(setV,vtype=GRB.BINARY,name="Multi") #(1-Y[i])*(1-I[i])

#constraints
# Y[i] +I[i] <= 1   (1)
for i in setV:
    Math_Model.addConstr((Yi[i]+ Ii[i]), sense=GRB.LESS_EQUAL, rhs=1, name="Bloklanan birey infected olamaz".format(i))

    # Mult[i] =  (1-Y[i])*(1-I[i])   (1)
for i in setV:
    Math_Model.addConstr(Multi[i], sense=GRB.EQUAL, rhs=(1-Yi[i])*(1-Ii[i]), name="Multi constraint".format(i))#constraints
# Y[i] +I[i] <= 1   (1)
for i in setV:
    Math_Model.addConstr((Yi[i]+ Ii[i]), sense=GRB.LESS_EQUAL, rhs=1, name="Bloklanan birey infected olamaz".format(i))

#(Σ(over j in V  w[i][j]*I[j]))*(1-Y[i])*(1-I[i]) <=  t[i]    for all i in S
for i in setS:
    Math_Model.addConstr(lhs=gp.quicksum(wij[i-1][j-1]*Ii[j] for j in setV)*Multi[i] , sense=GRB.LESS_EQUAL, rhs=ti[i-1],name="Birinci:Tresholdu geçen birey hasta olur.".format(i))
#(Σ(over j in V  w[i][j]*I[j]))*(1-Y[i]) >  t[i]*I[i]         for all i in S 
for i in setS:
    Math_Model.addConstr(lhs=gp.quicksum(wij[i-1][j-1]*Ii[j] for j in setV)*(1-Yi[i]) , sense=GRB.GREATER_EQUAL, rhs=ti[i-1]*Ii[i],name="İkinci:Tresholdu geçen birey hasta olur.".format(i))
#Σ(over i Y[i])<= c 

Math_Model.addConstr(lhs=gp.quicksum(Yi[i] for i in setV) , sense=GRB.LESS_EQUAL, rhs=Blocking_Capacity,name="Bloklama kapasitesinden fazla bloklama yapılamaz.".format(i))

#I[i]=1       for all i in V-S(P) 

for i in setP:
    Math_Model.addConstr(Ii[i], sense=GRB.EQUAL, rhs=1, name="Başlangıçta enfekte olanların I[i] parametresi bire eşittir.".format(i))
    
Math_Model.write("out.lp")
# asd.computeIIS()
# asd.write("model.ilp")

Math_Model.ModelSense = GRB.MINIMIZE
Math_Model.optimize()

#for v in Math_Model.getVars():
    #if(v.x>= -0.0001):
        #print('%s %g' % (v.varName, v.x))

vars = Math_Model.getVars()

adjacency_df['Blocked']=np.zeros((n,1))

adjacency_df['Ending_Infected']=np.zeros((n,1))


for i in range(1,n+1):
    adjacency_df.at[i,'Blocked']=vars[i-1].x
    adjacency_df.at[i,'Ending_Infected']=vars[i+n-1].x

Objective_Value = Math_Model.ObjVal
end_time = time.time()

adjacency_df.loc[(adjacency_df.loc[:,'IsRisky'] == 1) & (adjacency_df.loc[:,'Ending_Infected'] == 1) ,['Threshold','IsInfected','IsRisky','Blocked','Ending_Infected']]

adjacency_df['Blocked'].to_csv("Math_Model_NODE-n"+str(n)+'-m'+str(m)+'-OBJ'+str(Objective_Value)+".csv",index=False)       

Academic license - for non-commercial use only - expires 2021-06-09
Using license file C:\Users\utku\gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 211 rows, 300 columns and 510 nonzeros
Model fingerprint: 0xbadc93f4
Model has 280 quadratic constraints
Variable types: 0 continuous, 300 integer (300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e-03, 1e+00]
  QLMatrix range   [1e-03, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
  QRHS range       [1e-02, 1e+00]
Presolve removed 53 rows and 76 columns
Presolve time: 0.25s
Presolved: 562 rows, 584 columns, 2435 nonzeros
Variable types: 0 continuous, 584 integer (584 binary)
Found heuristic solution: objective 30.0000000
Found heuristic solution: objective 26.0000000

Root relaxation: objective 3.128350e+00, 414 iteration

In [3]:
print(end_time-start_time)

5.82062029838562


In [4]:
ati=adjacency_df['Blocked'].tolist()

In [5]:
adjacency_df.loc[adjacency_df['Ending_Infected']>0.5,'IsRisky'].sum()

13.0

In [6]:
adjacency_df

,1,2,3,4,5,6,7,8,9,10,...,96,97,98,99,100,Threshold,IsInfected,IsRisky,Blocked,Ending_Infected
1,0.000,0.0,0.0,0.0,0.257,0.0,0.0,0.0,0.0,0.0,...,0.138,0.0,0.0,0.0,0.0,0.733,0.0,1.0,0.0,1.0
2,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.107,0.0,0.0,-0.0,1.0
3,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.254,0.0,1.0,0.0,1.0
4,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.212,0.0,1.0,-0.0,0.0
5,0.257,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.654,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.138,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.058,0.0,1.0,0.0,1.0
97,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.957,0.0,0.0,-0.0,0.0
98,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.946,0.0,1.0,-0.0,0.0
99,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.469,1.0,0.0,0.0,1.0


In [7]:
sum(ati)

20.0